In [6]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvFCModelGenerator, ConvGAPModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
cifar10 = dset.CIFAR10('data//CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

In [7]:
NUM_VAL = 1024
loader_gpu = dload.get_loader_gpu(cifar_gpu, NUM_VAL)

model_gen = ConvFCModelGenerator((3, 32, 32), 10, conv_num = 6, lin_num = 2)
model_tmpl = model_gen.generate_model_tmpl()
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(model_tmpl)

Conv2d       (64, 16, 16)   (2, 2)   (2, 2)  
PReLU        (64, 16, 16)  
Conv2d       (128, 9, 9)    (4, 4)   (2, 2)  
PReLU        (128, 9, 9)   
MaxPool2d    (128, 5, 5)    (4, 4)   (2, 2)  
Conv2d       (192, 1, 1)    (5, 5)   (1, 1)  
LeakyReLU    (192, 1, 1)    (0.001) 
AvgPool2d    (192, 2, 2)    (2, 2)   (1, 1)  
Conv2d       (128, 1, 1)    (2, 2)   (3, 3)  
BatchNorm2d  (128, 1, 1)   
LeakyReLU    (128, 1, 1)    (0.1)   
MaxPool2d    (128, 2, 2)    (2, 2)   (1, 1)  
Conv2d       (64, 1, 1)     (3, 3)   (3, 3)  
ReLU         (64, 1, 1)    
BatchNorm2d  (64, 1, 1)    
LeakyReLU    (64, 1, 1)     (0.1)   
Flatten      64            
Linear       2048          
ReLU         2048          
Linear       10            



In [30]:
model_gen = ConvGAPModelGenerator((3, 32, 32), 10, conv_num = 6)
model_tmpl = model_gen.generate_model_tmpl()
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(model_tmpl)

Conv2d       (96, 29, 29)   (4, 4)   (1, 1)  
MaxPool2d    (96, 30, 30)   (2, 2)   (1, 1)  
BatchNorm2d  (96, 30, 30)  
AvgPool2d    (96, 28, 28)   (3, 3)   (1, 1)  
ReLU         (96, 28, 28)  
Conv2d       (128, 28, 28)  (7, 7)   (1, 1)  
LeakyReLU    (128, 28, 28)  (0.1)   
AvgPool2d    (128, 28, 28)  (3, 3)   (1, 1)  
GlobalAvgPool2d (128, 1, 1)   
Flatten      128           
Linear       10            



In [32]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, NUM_VAL, 1024)
# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 26.5s 
val_acc: 0.345, train_acc: 0.330


In [15]:
print(len(cifar10))

50000


In [23]:
loader = dload.get_loader(cifar10, NUM_VAL, 1024)
batch = next(iter(loader_gpu['train']))
print(len(batch[0]))
print(type(batch[0]))


1024
<class 'torch.Tensor'>


In [ ]:
# t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
#                                           train = True,
#                                           loader = loader,
#                                           milestones = [],
#                                           num_epoch = 1,
#                                           verbose = True)